In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()
warnings.filterwarnings('ignore')

In [3]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding
from IPython.display import Markdown, display

In [4]:
from llama_index.readers.web import SimpleWebPageReader
import requests
from llama_index.core import Document

class CustomWebPageReader(SimpleWebPageReader):
    """
    Many websites, including Investopedia, require headers like User-Agent to be set in the request to return the correct content.
    To fix this, we'll modify the load_data method in the SimpleWebPageReader class to include appropriate headers.
    """
    
    def load_data(self, urls):
        if not isinstance(urls, list):
            raise ValueError("urls must be a list of strings.")
        documents = []
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        for url in urls:
            response = requests.get(url, headers=headers).text
            if self.html_to_text:
                import html2text
                response = html2text.html2text(response)

            metadata = None
            if self._metadata_fn is not None:
                metadata = self._metadata_fn(url)

            documents.append(Document(text=response, id_=url, metadata=metadata or {}))

        return documents

links = [
    "https://www.investopedia.com/terms/s/stockmarket.asp",
    "https://www.investopedia.com/ask/answers/difference-between-options-and-futures/",
    "https://www.investopedia.com/financial-edge/0411/5-essential-things-you-need-to-know-about-every-stock-you-buy.aspx",
    "https://www.investopedia.com/articles/fundamental/04/063004.asp",
    "https://www.investopedia.com/terms/t/technicalanalysis.asp",
    "https://www.investopedia.com/terms/i/ichimoku-cloud.asp",
    "https://www.investopedia.com/terms/a/aroon.asp",  
    "https://www.investopedia.com/terms/b/bollingerbands.asp",
    "https://www.investopedia.com/articles/forex/05/macddiverge.asp",
    "https://www.investopedia.com/terms/a/accumulationdistribution.asp",
    "https://www.investopedia.com/terms/s/stochasticoscillator.asp",
    "https://www.investopedia.com/terms/s/stochrsi.asp",
    "https://www.investopedia.com/terms/p/price-earningsratio.asp",
    "https://www.investopedia.com/terms/p/price-to-bookratio.asp",
    "https://www.investopedia.com/terms/p/price-to-salesratio.asp",
    "https://www.investopedia.com/terms/q/quickratio.asp"
]

In [5]:
docs = CustomWebPageReader(
    html_to_text=True
).load_data(urls=links)

In [6]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

client = qdrant_client.QdrantClient("http://localhost:6333")
aclient = qdrant_client.AsyncQdrantClient("http://localhost:6333")

if client.collection_exists("investopedia"):
    client.delete_collection("investopedia")

vector_store = QdrantVectorStore(
    collection_name="investopedia",
    client=client,
    aclient=aclient,
    fastembed_sparse_model="Qdrant/bm42-all-minilm-l6-v2-attentions",
)

Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
Settings.llm = BedrockConverse(
    model = "anthropic.claude-3-5-sonnet-20240620-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name = "us-east-1"
)
Settings.embed_model = BedrockEmbedding(
    model = "amazon.titan-embed-text-v1",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)

In [8]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context
)

Test qdrant client

In [9]:
query_engine = index.as_query_engine(similarity_top_k=4, sparse_top_k=10)

response = query_engine.query("What is the Aroon indicator?")

display(Markdown(f"<b>{response}</b>"))

<b>The Aroon indicator is a technical analysis tool used to identify changes in price trends and assess the strength of those trends. It consists of two main components:

1. The AroonUp line, which measures the time since the last 25-period high
2. The AroonDown line, which measures the time since the last 25-period low

These lines fluctuate between 0 and 100, with higher values indicating stronger trends. The indicator helps traders determine whether an asset is in an uptrend, downtrend, or consolidating.

Key points about the Aroon indicator:

- Readings above 50 suggest a high or low occurred within the last 12.5 periods
- Values near 100 indicate a very recent high or low
- Crossovers between the AroonUp and AroonDown lines can signal potential trend changes
- When both lines are below 50, it may indicate price consolidation

Traders use the Aroon indicator to identify trend strength, potential reversals, and entry or exit points in the market. However, it's generally recommended to use this indicator in conjunction with other technical analysis tools for more comprehensive trading decisions.</b>

In [11]:
def load_index(qdrant_vector_store: QdrantVectorStore,
               embed_model):
    return VectorStoreIndex.from_vector_store(
        qdrant_vector_store,
        embed_model
    ) 
    
index = load_index(vector_store, Settings.embed_model)
query_engine = index.as_query_engine(similarity_top_k=4, sparse_top_k=10)
response = query_engine.query("What is the Aroon indicator?")

display(Markdown(f"<b>{response}</b>"))


<b>The Aroon indicator is a technical analysis tool used to identify changes in price trends and assess the strength of those trends. It consists of two main components:

1. The AroonUp line
2. The AroonDown line

These lines measure the time between highs and lows over a specified period, typically 25 periods. The indicator works on the principle that strong uptrends will regularly see new highs, while strong downtrends will regularly see new lows.

The Aroon indicator ranges from 0 to 100. Readings above 50 suggest a high or low occurred within the last 12.5 periods, while readings near 100 indicate a very recent high or low.

Traders use the Aroon indicator to:

1. Identify trend changes
2. Assess trend strength
3. Spot potential buy or sell signals through crossovers of the AroonUp and AroonDown lines

When the AroonUp line is above the AroonDown line, it suggests bullish price behavior. Conversely, when the AroonDown line is above the AroonUp line, it indicates bearish price behavior.

The Aroon indicator was developed by Tushar Chande and is considered useful for analyzing trend changes and strength in various financial markets.</b>

In [12]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import (
    QueryEngineTool,
    ToolMetadata
)

query_engine_tool = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata = ToolMetadata(
            name="investopedia_tool",
            description = (
                "Provides conceptual information relating to the stock market, "
                "stock market instruments such as options and futures, "
                "as well as conceptual understanding for metrics used in "
                "technical analysis and fundamental analysis."
            )
        )
    )
]

agent = ReActAgent.from_tools(
    tools = query_engine_tool,
    verbose = True
)

response = agent.chat(
    "What are the advantages and disadvantages of the MACD?"
)

display(
    Markdown(
        f"<b>{response}</b>"
    )
)

> Running step 89f0a603-99ce-4aab-b81e-dcfd244899db. Step input: What are the advantages and disadvantages of the MACD?
Thought: The current language of the user is: English. To answer this question about the MACD (Moving Average Convergence Divergence), I need to gather information about its advantages and disadvantages. I'll use the investopedia_tool to get this information.
Action: investopedia_tool
Action Input: {'input': 'MACD advantages and disadvantages'}
Observation: Thought: I can answer this question using the information provided in the context. I'll summarize the advantages and disadvantages of MACD.

Answer: 

The Moving Average Convergence Divergence (MACD) indicator has several advantages and disadvantages:

Advantages:
1. Trend-following: MACD is effective at identifying trends and momentum in price movements.
2. Versatility: It can be used in various markets, including stocks, bonds, commodities, and forex.
3. Multiple strategies: Traders can employ different MACD stra

<b>The Moving Average Convergence Divergence (MACD) indicator has several advantages and disadvantages:

Advantages:
1. Trend identification: MACD is excellent at identifying trends and momentum in price movements, helping traders spot potential entry and exit points.
2. Versatility: It can be applied to various financial markets, including stocks, bonds, commodities, and forex.
3. Multiple strategies: Traders can use different MACD strategies, such as histogram analysis, crossover strategy, and zero-cross strategy, providing flexibility in trading approaches.
4. Complementary tool: MACD works well in combination with other technical indicators like RSI and SMA, allowing for more comprehensive market analysis.

Disadvantages:
1. False signals: MACD can sometimes produce false reversal signals, which may lead to losses if traders act on them without additional confirmation.
2. Less effective in sideways markets: In range-bound or consolidating markets, MACD tends to drift towards the zero line, making it less reliable for trading decisions.
3. Lagging indicator: The MACD zero-cross is a lagging indicator, which means traders might enter positions later than optimal, potentially missing out on some profits.
4. Complexity: The MACD's multiple components (two moving averages and a histogram) can be challenging for novice traders to interpret correctly, leading to potential misuse.

It's important to remember that while MACD is a popular and useful technical analysis tool, it should be used in conjunction with other analysis methods and proper risk management strategies for more effective trading decisions.</b>